## American filmographies from wikipedia

- Scrape link : https://en.wikipedia.org/w/index.php?title=Category:American_filmographies&pageuntil=Hayden%2C+Sterling%0ASterling+Hayden+filmography#mw-pages
- get Actors/Directors name url
- from each url, get film title, Release Year, role

## Process Note
- get the page with `request`
- Use `Bs4` for webpage Process
- Collect the info
- Create the func
- Create `CSV` with that info

## Scrape on a single page

- page url - https://en.wikipedia.org/wiki/Christian_Bale_filmography

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


In [2]:
page_url = 'https://en.wikipedia.org/wiki/List_of_Amy_Adams_performances'

In [3]:
resp = requests.get(page_url)

In [4]:
html_page = BeautifulSoup(resp.text,'html.parser')

In [5]:
table_tag = html_page.select('table.wikitable')[0]

In [6]:
# table_tag

In [7]:
columns = table_tag.find('tbody').find_all('th')[:3]
columns_name = [c.text.strip().lower() for c in columns]
columns_name[columns_name=='year'] = 'release_year'
columns_name

['release_year', 'title', 'role']

In [8]:
table_df = pd.read_html(str(table_tag))

table_df=np.array(table_df)
table_df = table_df[:,:,:3].copy()
table_df =pd.DataFrame(table_df.reshape(-1,3),columns=columns_name)
# table_df.iloc[:,:3]
table_df

,release_year,title,role
0,1999,Drop Dead Gorgeous,Leslie Miller
1,2000,Psycho Beach Party,Marvel Ann
2,2000,The Chromium Hook,Jill Royaltuber
3,2000,Cruel Intentions 2,Kathryn Merteuil
4,2002,The Slaughter Rule,Doreen
5,2002,Pumpkin,Alex
6,2002,Serving Sara,Kate
7,2002,Catch Me If You Can,Brenda Strong
8,2004,The Last Run,Alexis
9,2005,The Wedding Date,Amy Ellis


## scrape all pages of A-H
-url: https://en.wikipedia.org/w/index.php?title=Category:American_filmographies&pageuntil=Hayden%2C+Sterling%0ASterling+Hayden+filmography#mw-pages

func for each name

In [9]:
def get_column_name(table_tag):
    columns = table_tag.find('tbody').find_all('th')[:3]
    columns_name = [c.text.strip().lower() for c in columns]
    columns_name[columns_name=='year'] = 'release_year'
    return columns_name

def get_table(table_tag,columns_name):
    table_df = pd.read_html(str(table_tag))
    table_df=np.array(table_df)
    table_df = table_df[:,:,:3].copy()
    table_df =pd.DataFrame(table_df.reshape(-1,3),columns=columns_name)
    return table_df

def scrape_each_url(page_url):
    resp = requests.get(page_url)
    html_page = BeautifulSoup(resp.text,'html.parser')
    table_tag = html_page.select('table.wikitable')[0]

    columns_name = get_column_name(table_tag)

    table_df = get_table(table_tag,columns_name)

    return pd.DataFrame(table_df)
    


funcs for get A-Z page list
- base url - https://en.wikipedia.org
- get `name` and `url` with `scrape_topics`
- call each `url`

In [10]:
def get_actor_name(html_doc):
    actor_name_tag = html_doc.find('div',{'class': 'mw-category'}).find_all('li')
    actor_name_list = [actor.text.strip() for actor in actor_name_tag]
    return actor_name_list

def get_actor_url(html_doc):
    actor_url_tag = html_doc.find('div',{'class': 'mw-category'}).find_all('a')
    actor_url_list = ['https://en.wikipedia.org'+url['href'] for url in actor_url_tag]
    return actor_url_list

def scrape_all_category():
    page_url = ' https://en.wikipedia.org/w/index.php?title=Category:American_filmographies&pageuntil=Hayden%2C+Sterling%0ASterling+Hayden+filmography#mw-pages'
    resp = requests.get(page_url)
    if resp.status_code!=200:
        raise Exception(f'Failed to load{page_url}')
    html_doc =  BeautifulSoup(resp.text,'html.parser')

    actor_dict = {
        'actor_name' : get_actor_name(html_doc),
        'url' : get_actor_url(html_doc),
    }
    return pd.DataFrame(actor_dict)

def scrape_each_category():
    actor_df = scrape_all_category()
    print(actor_df['actor_name'][5])
    each_df = scrape_each_url(actor_df['url'][5])
    return each_df
    # for _,row in actor_df.iterrows():
    #     print(row['actor_name'])
    #     scrape_each_url(row['url'])

    

In [11]:
scrape_each_category()

Paul Thomas Anderson filmography


,release_year,year,budget
0,Hard Eight,1996,"$3,000,000"
1,Boogie Nights,1997,"$15,000,000"
2,Magnolia,1999,"$37,000,000"
3,Punch-Drunk Love,2002,"$25,000,000"
4,There Will Be Blood,2007,"$25,000,000"
5,The Master,2012,"$32,000,000"
6,Inherent Vice,2014,"$20,000,000"
7,Phantom Thread,2017,"$35,000,000"
8,Soggy Bottom,2021,"$40,000,000"


## More scrape
Url for A-Z 
- A-H :
https://en.wikipedia.org/w/index.php?title=Category:American_filmographies&pageuntil=Hayden%2C+Sterling%0ASterling+Hayden+filmography#mw-pages

- H-S :
https://en.wikipedia.org/w/index.php?title=Category:American_filmographies&pagefrom=Hayden%2C+Sterling%0ASterling+Hayden+filmography#mw-pages

- S-Z:
https://en.wikipedia.org/w/index.php?title=Category:American_filmographies&pagefrom=Shepard%2C+Jan%0AJan+Shepard+filmography#mw-pages


## optional 


find the title

In [12]:
# row_tag = table_tag.find('tbody').find_all('tr')
# title = []
# for tr in row_tag:
#     th = tr.find_all('th')
#     th_text = [t.text.strip() for t in th]
#     title.append(th_text)
# columns_name = title.pop(0)
# title

find name/url

In [13]:
# page_url = ' https://en.wikipedia.org/w/index.php?title=Category:American_filmographies&pageuntil=Hayden%2C+Sterling%0ASterling+Hayden+filmography#mw-pages'
# reqs = requests.get(page_url)
# if reqs.status_code!=200:
#     raise Exception(f'Failed to load{page_url}')
# html_doc =  BeautifulSoup(reqs.text,'html.parser')
# # actor_name_tag = html_doc.find('div',{'class': 'mw-category'}).find_all('li')
# # name_list = [actor.text.strip() for actor in actor_name_tag]
# actor_url_tag = html_doc.find('div',{'class': 'mw-category'}).find_all('a')
# actor_url_list = ['https://en.wikipedia.org'+url['href'] for url in actor_url_tag]
# actor_url_list[:5]